### P0 Parsing Tests

**Emil Sekerinski, McMaster University, revised February 2022**

In [1]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString

Importing Jupyter notebook from P0.ipynb
Importing Jupyter notebook from SC.ipynb
Importing Jupyter notebook from ST.ipynb


In [2]:
def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

In [3]:
def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))

In [10]:
compileString("""
program p
    var x: set[0..1]
    var i : integer
    var m : integer
    var j : integer
    var n : integer
    x := {i+2 for i in range 5 if i > 2}
    m := 1
    j := 3
    n := 5
    if m ∈ x then write(m)
    if j ∈ x then write(j)
    if n ∈ x then write(n)
""", "example.wat")

In [11]:
!wat2wasm example.wat

In [12]:
runpywasm("example.wasm")

5


In [19]:
compileString("""
program p
    var x: set[0..1]
    var i : integer
    var m : integer
    var j : integer
    var n : integer
    x := {i for i in range 5 if i > 2}
    m := 1
    j := 3
    n := 5
    if m ∈ x then write(m)
    if j ∈ x then write(j)
    if n ∈ x then write(n)
""", "example.wat")

In [17]:
!wat2wasm example.wat

In [18]:
runpywasm("example.wasm")

3


In [13]:
compileString("""
program p
    var x: set[0..1]
    var i : integer
    var m : integer
    var j : integer
    var n : integer
    x := {i for i in range 5}
    m := 1
    j := 3
    n := 5
    if m ∈ x then write(m)
    if j ∈ x then write(j)
    if n ∈ x then write(n)
""", "example.wat")

In [14]:
!wat2wasm example.wat

In [15]:
runpywasm("example.wasm")

1
3


In [20]:
compileString("""
program p
    var y: [0..1] → integer
    var i: integer
    y := [i × 5 for i in range 5 if i > 2]
    write(y[0])
    write(y[1])
""", "example.wat")

In [21]:
!wat2wasm example.wat

In [22]:
runpywasm("example.wasm")

15
20


In [23]:
compileString("""
program p
    var y: [0..5] → integer
    var i: integer
    y := [i×5 for i in range 5]
    write(y[0])
    write(y[1])
    write(y[2])
    write(y[3])
    write(y[4])
""", "example.wat")

In [24]:
!wat2wasm example.wat

In [25]:
runpywasm("example.wasm")

0
5
10
15
20


In [26]:
compileString("""
program p
    var i: boolean
    i := found 8 [1..10]
    write(i)
""", "example.wat")

In [27]:
!wat2wasm example.wat

In [28]:
runpywasm("example.wasm")

1


In [29]:
compileString("""
program p
    var y: [0..5] → integer
    var i: integer
    y := sorted [5,3,20,1,4,0]
    write(y[0])
    write(y[1])
    write(y[2])
    write(y[3])
    write(y[4])
    write(y[5])
""", "example.wat")

In [30]:
!wat2wasm example.wat

In [31]:
runpywasm("example.wasm")

0
1
3
4
5
20


In [32]:
compileString("""
program p
    var x: [0..5] → integer
    var i: integer
    x := [6•i×i]
    write(x[0])
    write(x[1])
    write(x[2])
    write(x[3])
    write(x[4])
    write(x[5])
""", "example.wat")

In [33]:
!wat2wasm example.wat

In [34]:
runpywasm("example.wasm")

0
1
4
9
16
25


In [35]:
compileString("""
program p
    var x: set[0..1]
    var i : integer
    var m : integer
    var j : integer
    var n : integer
    x := {5 | i mod 2 = 1 • i}
    m := 1
    j := 3
    n := 5
    if m ∈ x then write(m)
    if j ∈ x then write(j)
    if n ∈ x then write(n)
""", "example.wat")

In [36]:
!wat2wasm example.wat

In [37]:
runpywasm("example.wasm")

1
3


Procedure `compileerr(s)` returns an empty string if compiling `s` succeeds or the error message produced while compiling; the error message is also printed. The procedure is used here to test parsing.

In [ ]:
def compileerr(s):
    try: compileString(s); return ''
    except Exception as e:
        print(e); return str(e)

#### Error "] expected"

In [ ]:
assert "] expected" in compileerr("""
var a: [1..10] → integer
var x: integer
program p
    x := a[4
""")

#### Error ") expected"

In [ ]:
assert ") expected" in compileerr("""
program p
  var x: integer
    x := (5
""")

#### Error "expression expected"

In [ ]:
assert "expression expected" in compileerr("""
program p
  var x: integer
    x := +
""")

#### Error "new line expected"

In [ ]:
assert "new line expected" in compileerr("""
program p
  write(5) write(7)
""")

#### Error "dedent or new line expected"

In [ ]:
assert "dedent or new line expected" in compileerr("""
program p
  write(5)
    write(7)
""")

#### Error "indented statement expected"

In [ ]:
assert "indented statement expected" in compileerr("""
procedure p()
  if 3 > 4 then
writeln()
""")

#### Error "variable for result expected"

In [ ]:
assert "variable for result expected" in compileerr("""
program p
  read()
""")

#### Error ":= or ← expected"

In [ ]:
assert ":= or ← expected" in compileerr("""
var a: [5 .. 7] → integer
program p
  var b: boolean
    a[5] +
""")

#### Error "'(' expected"

In [ ]:
assert "'(' expected" in compileerr("""
program p
  writeln
""")

#### Error "')' expected"

In [ ]:
assert "')' expected" in compileerr("""
program p
  writeln(
""")

#### Error "'then' expected"

In [ ]:
assert "'then' expected" in compileerr("""
program p
  if true write(5)
""")

#### Error "'do' expected"

In [ ]:
assert "'do' expected" in compileerr("""
program p
  while true write(5)
""")

#### Error "statement expected"

In [ ]:
assert "statement expected" in compileerr("""
program p
  write(3); const c = 5
""")

#### Error "'..' expected"

In [ ]:
assert "'..' expected" in compileerr("""
var a: [5 → integer
program p
  writeln()
""")

#### Error "']' expected"

In [ ]:
assert "']' expected" in compileerr("""
var a: [5..7 → integer
program p
  writeln()
""")

#### Error "'→' expected"

In [ ]:
assert "'→' expected" in compileerr("""
var a: [3 .. 7] integer
program p
  writeln()
""")

#### Error "type expected"

In [ ]:
assert "type expected" in compileerr("""
program p
  var x: if
""")

#### Error "identifier expected"

In [ ]:
assert "identifier expected" in compileerr("""
program p
  var if: integer
""")

#### Error "identifier expected"

In [ ]:
assert "identifier expected" in compileerr("""
program p
  var if: integer
""")

#### Error "identifier expected"

In [ ]:
assert "identifier expected" in compileerr("""
program p
  var x, if: integer
""")


#### Error "':' expected"

In [ ]:
assert "':' expected" in compileerr("""
program p
  var x integer
""")

#### Error "identifier expected"

In [ ]:
assert "identifier expected" in compileerr("""
program p
  var i, j: integer, if: boolean
""")

#### Error "identifier expected"

In [ ]:
assert "identifier expected" in compileerr("""
program p
  var i, j: integer, b, if: boolean
""")

#### Error "constant name expected"

In [ ]:
assert "constant name expected" in compileerr("""
program p
  const 5 = 7
  write(3)
""")

#### Error "= expected"

In [ ]:
assert "= expected" in compileerr("""
program p
  const c: 5
  write(5)
""")

#### Error "type name expected"

In [ ]:
assert "type name expected" in compileerr("""
program p
  type 5 = integer
  write(3)
""")

#### Error "= expected"

In [ ]:
assert "= expected" in compileerr("""
program p
  type T: integer
  writeln()
""")

#### Error  "procedure name expected"

In [ ]:
assert "procedure name expected" in compileerr("""
procedure
  writeln()
program p
  writeln()
""")

#### Error  "( expected"

In [ ]:
assert "( expected" in compileerr("""
procedure q
  writeln()
program p
  writeln()
""")

#### Error  ") expected"

In [ ]:
assert ") expected" in compileerr("""
procedure q(
  writeln()
program p
  writeln()
""")

#### Error  "( expected"

In [ ]:
assert "( expected" in compileerr("""
procedure q(x: integer) → boolean
  writeln()
program p
  writeln()
""")

#### Error  ") expected"

In [ ]:
assert ") expected" in compileerr("""
procedure q(x: integer) → (y: boolean
  writeln()
program p
  writeln()
""")

#### Error  "indent expected"

In [ ]:
assert "indent expected" in compileerr("""
program p
writeln()
""")

#### Error "dedent or new line expected"

In [ ]:
assert "dedent or new line expected" in compileerr("""
program p
  const c = 5
    writeln()
      writeln()
""")

#### Error "statement expected"

In [ ]:
assert "statement expected" in compileerr("""
program p
  program q
""")

#### Error "dedent or new line expected"

In [ ]:
assert "dedent or new line expected" in compileerr("""
program p
  writeln()
    writeln()
""")

#### Error "'program' expected"

In [ ]:
assert "'program' expected" in compileerr("""
var x: integer
""")

#### Error "program name expected"

In [ ]:
assert "program name expected" in compileerr("""
program
  writeln()
""")

#### Multiple Indentations

In [ ]:
assert compileString("""
procedure q()
  var b: boolean
    b := true
    if b then write(3)
    else write(5)
    if ¬b then write(5)
    else if b then write(7)
    else write(9)
    while b do
      if b then
        b := false; write(1)
program p
  if 3 > 4 then writeln() else
    q()
""") == """\
(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(func $q  
(local $b i32)
(local $0 i32)
i32.const 1
local.set $b
local.get $b
if
i32.const 3
call $write
else
i32.const 5
call $write
end
local.get $b
i32.eqz
if
i32.const 5
call $write
else
local.get $b
if
i32.const 7
call $write
else
i32.const 9
call $write
end
end
loop
local.get $b
if
local.get $b
if
i32.const 0
local.set $b
i32.const 1
call $write
end
br 1
end
end
)
(global $_memsize (mut i32) i32.const 0)
(func $program
(local $0 i32)
i32.const 0
if
call $writeln
else
call $q
end
)
(memory 1)
(start $program)
)"""